## Laboratorio 4: Image colour space for object segmentation

Elaborado por: Oscar Omar Martínez Lujano 

Matrícula: 352228  
Carrera: ITR  
Fecha: 2019-02-17  


### Introduction

Thus far, we have seen how to convert a BGR-colour image into a grayscale image using the OpenCV method cv2.cvtColour(frame, cv2.COLOR_BGR2GRAY). This converted a 3-channel image into a single-channel greyscale image. Similarly to the greyscale colour space, there exists a vast number of other colour spaces being the RBG (Red, Blue, Green), HSV (Hue, Saturation, Value), HLS (Hue, Lightness, Saturation), and the LAB (Lightness, Green-to-Magenta, Blue-to-Yellow) colour spaces amongst the most widely used. 

Depending on the computer vision application being developed, and assuming your vision system captures colour images, you may prefer to process the input images using either the RGB colour space or the Grey colour space. For instance, if you are interested in detecting objects with particular geometric shapes such as circles, triangles, lines, etc., working on the RGB colour space may not add any further information, compared with working on the Grey colour space. This is because detecting the edges of an object is not a function of the object colour; on the contrary, this may make the computer vision algorithm computationally slower, as processing three-channel images always takes more time than processing single-channel images.

In a different computer vision application, you may be interested in finding the lane lines on a highway. In this case, you may prefer to integrate colour information into your algorithm as you know that lanes are regularly painted in black or white colour. Furthermore, you also know that lane lines are also painted in either white or yellow colour. Therefore, fusing colour information may facilitate the detection and tracking of lane lines on a highway. Another application where you may wish to integrate colour information into your algorithm is when being interested in traffic lights and skin tone detection. For the former application, it is quite likely that the traffic lights will be yellow, green or red in most cases; whereas for the latter application, depending on factors such as geographical location, ethnic group, gender, age, etc., you may be able to develop a computer vision application for skin detection.

We now have a brief understanding on what short of computer vision applications may rely on the use of the RBG or the Grey color space for object detection/segmentation. We also noticed that OpenCV has 274 different colour spaces available for image processing; thus, why there exits so many colour spaces?, when would the HLS colour space be preferred in a computer vision application?. Going back to the automated highway lane line detection task, it turns out that aspects such as lighting conditions highly affect the performance of an algorithm working with RGB colour images. In that situation, we may prefer to convert our RGB image into a different colour space, such as the HSL (hue, saturation, lightness). This has proven to be more robust even when different lighting conditions are present. The figures below show an example of a lane lines segmentation when using the RGB and the HLS colour spaces, respectively. As can be seen, we may consider using the R and G channels to automatically extract the yellow line from the colour image; however, as indicated by the red circle, these two channels are affected by the shadow produced by the high tree located ahead of the road. This may limit the algorithm to detect and track lane lines at short distances. On the contrary, the second image shows how this line could be better detected using either low intensity values from the H channel or high intensity values from the S channel, as these two channels seem to be robust to shadows or to different lighting conditions. Being able to detect the road lane lines at a farther distance makes the vehicle able to respond to an event of emergency accordingly.

### Objectives

In this lab, we will learn about image colour space, particularly about the HSV (Hue, Saturation, Value) and HLS (Hue, Saturation, Lightness) colour maps. You will use this colour space to segment an object in a video sequence, this object can be either a fish swimming in a fishbolw, lane lines on a road, or any other object that can be segmented by using its corresponding HSV components.

### Laboratory Requirements
The software, hardware and programming tools required for this lab are listed below:

    - Laptop or Raspberry Pi with WiFi connection capabilities
    - Jupyter Notebook
    - Python >= 3.5.2
    - OpenCV 3.2
    - Git
    - GitHub account
    - Markdown editor



### Procedure

In this code, you will use the OpenCV method hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) to convert the current frame from RGB to HSV. Once you have the hsv image, you will use the method mask = cv2.inRange(hsv, lower_blue, upper_blue) to create a mask image. This will be a binary image with high pixel intensity values in those regions where the colour of interest appears in the current frame. In order to generate a new image that contains the segmented blue colour objects, it is necessary to perform an AND bit-wise operation between the current frame and the mask image, so that only those pixels with high intensity values in the mask image are visualised in the new image.

The next images were used to take out hsv_min and hsv_max with GIMP help:

<img src="Figs/mardigra.png" width="400" alt="Combined Image" />
<img src="Figs/volti-01.png" width="400" alt="Combined Image" />



### a. Importación de librerías

The following libraries are used in the codes of this lab.


- ```cv2```: Implementa una gran variedad de algorítmos de procesamiento de imágenes y visión computacional.
- ```numpy:``` Crea y manipula listas, análisis numérico, etc.
- `matplotlib.pyplot:` Produce publicaciones con figuras de calidad en una variedad de formatos de copia impresa y entornos interactivos en todas las plataformas.

In [2]:
# importa librerías estandar
import numpy as numpy
import cv2
import matplotlib.pyplot as plt

### b. Object segmentation using the HSV (Hue, Saturation, Value) colour information

The code below shows you how to segment (detect) the following objects:

    - A fish swimming in a fishbowl
    - White lane lines on a running track
    - Red lane on a running track
    - Hawaiian flowers - blue objects

    
- `cv2.bitwise_and(src1, src2[, dst[, mask]])`
    - Calculates the per-element bit-wise conjunction of two arrays or an array and a scalar.
    - src1 – first input array or a scalar..
    - src2 – second input array or a scalar.
    - src – single input array..
    - value – scalar value.
    - **returns** - array that has the same size and type as the input arrays.
    
- `cv.inRange(src, lowerb, upperb[, dst])`
    - Checks if array elements lie between the elements of two other arrays. 
    - src - first input array.
    - lowerb - inclusive lower boundary array or a scalar. 
    - upperb - inclusive upper boundary array or a scalar. 
    - **returns**: 	output array of the same size as src and CV_8U type. 



Información obtenida de:
- https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#bitwise-and
- https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981


In [17]:
"""
    object-segmentation-using-hsv-colour-space.py

    add a description of your code here

    author: add your fullname
    date created: add this info
    universidad de monterrey
"""

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2


# config video object
def config_video_source(source_video_file):

    # initialise a video capture object
    cap = cv2.VideoCapture(source_video_file)

    # check that the videocapture object was successfully created
    if(not cap.isOpened()):
        print("Error opening video source")
        exit()

    # return videocapture object
    return cap


# process video
def process_video(cap, hsv_min=(20, 20, 29), hsv_max=(40, 200, 200)):


    # create new windows for visualisation purposes
    cv2.namedWindow('input video', cv2.WINDOW_NORMAL)
    cv2.namedWindow('segmented object', cv2.WINDOW_NORMAL)

    # main loop
    while(cap.isOpened()):

        # grab current frame
        ret, frame = cap.read()

        # verify that frame was properly captured
        if ret == False:
            print("ERROR: current frame could not be read")            
            break

        # convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # threshold the hsv image so that only blue pixels are kept
        mask = cv2.inRange(hsv, hsv_min, hsv_max)

        # AND-bitwise operation between the mask and input images
        segmented_objects = cv2.bitwise_and(frame, frame, mask=mask)

        # visualise current frame
        cv2.imshow('input video',frame)

        # visualise segmented blue object
        cv2.imshow('segmented object', segmented_objects)

        # Display the resulting frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite('volti-swimming.png', frame)
            cv2.imwrite('volti-swimming-segmented.png', segmented_objects)
            break


# free memory and close open windows
def free_memory_amd_close_windows(cap):

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


# run pipeline
def run_pipeline(dataset_number=1):


    # select video sequence
    if(dataset_number==1):
        # segment volti
        cap = config_video_source(source_video_file="volti-01.mp4")
        process_video(cap, hsv_min=(115, 30, 0), hsv_max=(190, 250, 250))           

    elif(dataset_number==2):
        # segment white lane lines
        cap = config_video_source(source_video_file="running-track.mp4")
        process_video(cap, hsv_min=(130, 0, 100), hsv_max=(160, 255, 255))

    elif(dataset_number==3):
        # segment red road
        cap = config_video_source(source_video_file="running-track.mp4")
        process_video(cap, hsv_min=(160, 85, 85), hsv_max=(180, 200, 200))

    elif(dataset_number==4):
        # segment blue objects
        cap = config_video_source(source_video_file="mardigrass.mp4")
        process_video(cap, hsv_min=(95, 40, 100), hsv_max=(110, 255, 255))
    else:
        # enter a valid option
        print("Please, enter a valid option mate! [1-4]")
        exit()

    # free memory and close windows
    free_memory_amd_close_windows(cap)


# main function
def main():    
    run_pipeline(dataset_number = 4)


if __name__=='__main__':
    main()

TO COMPLETE THIS SECTION... 


    - Use the mardigrass.mp4 video sequence to segment multiple objects. In particular, your code should be able to detect and display the segmentation of blue, red, yellow and violet colour objects.
    - Use the volti-01.mp4 sequence to segment both the fish (red-ish) and the fishbowl floor (blue-ish).


In [16]:
"""
    object-segmentation-using-hsv-colour-space.py

    add a description of your code here

    author: add your fullname
    date created: add this info
    universidad de monterrey
"""

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2


# config video object
def config_video_source(source_video_file):

    # initialise a video capture object
    cap = cv2.VideoCapture(source_video_file)

    # check that the videocapture object was successfully created
    if(not cap.isOpened()):
        print("Error opening video source")
        exit()

    # return videocapture object
    return cap


# process video
def process_video(cap, hsv_min=(20, 20, 29), hsv_max=(40, 200, 200)):


    # create new windows for visualisation purposes
    cv2.namedWindow('input video', cv2.WINDOW_NORMAL)
    cv2.namedWindow('segmented object', cv2.WINDOW_NORMAL)

    # main loop
    while(cap.isOpened()):

        # grab current frame
        ret, frame = cap.read()

        # verify that frame was properly captured
        if ret == False:
            print("ERROR: current frame could not be read")            
            break

        # convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # threshold the hsv image so that only blue pixels are kept
        mask = cv2.inRange(hsv, hsv_min, hsv_max)

        # AND-bitwise operation between the mask and input images
        segmented_objects = cv2.bitwise_and(frame, frame, mask=mask)

        # visualise current frame
        cv2.imshow('input video',frame)

        # visualise segmented blue object
        cv2.imshow('segmented object', segmented_objects)

        # Display the resulting frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite('volti-swimming.png', frame)
            cv2.imwrite('volti-swimming-segmented.png', segmented_objects)
            break


# free memory and close open windows
def free_memory_amd_close_windows(cap):

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


# run pipeline
def run_pipeline(dataset_number=1):


    # select video sequence
    if(dataset_number==1):
        # blue
        cap = config_video_source(source_video_file="mardigrass.mp4")
        process_video(cap, hsv_min=(95, 40, 100), hsv_max=(110, 255, 255))
        # red 
        cap = config_video_source(source_video_file="mardigrass.mp4")
        process_video(cap, hsv_min=(160, 65, 85), hsv_max=(180, 200, 200))
        # yellow 
        cap = config_video_source(source_video_file="mardigrass.mp4")
        process_video(cap, hsv_min=(17, 40, 100), hsv_max=(32, 255, 255))
        # violet
        cap = config_video_source(source_video_file="mardigrass.mp4")
        process_video(cap, hsv_min=(125, 40, 100), hsv_max=(140, 255, 255))
                                                           
    elif(dataset_number==2):
        # blue ish
        cap = config_video_source(source_video_file="volti-01.mp4")
        process_video(cap, hsv_min=(115, 30, 0), hsv_max=(190, 250, 250))
        #red ish 
        cap = config_video_source(source_video_file="volti-01.mp4")
        process_video(cap, hsv_min=(93, 30, 0), hsv_max=(108, 250, 250))
        

    else:
        # enter a valid option
        print("Please, enter a valid option mate! [1-4]")
        exit()

    # free memory and close windows
    free_memory_amd_close_windows(cap)


# main function
def main():    
    run_pipeline(dataset_number = 2)


if __name__=='__main__':
    main()

### Conclusiones


_Yo declaro, que he realizado este Laboratorio 3 con integridad académica_